In [ ]:
import pandas as pd
import seaborn as sb

In [ ]:
df = sb.load_dataset('diamonds')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sb.pairplot(df)

In [ ]:
df.query("x==0 or y==0 or z==0 or y>20 or z>15")

In [ ]:
df.drop(df.query("x==0 or y==0 or z==0 or y>20 or z>6 or z<=2").index, inplace=True)

In [ ]:
sb.pairplot(df)

In [ ]:
df.describe()

In [ ]:
len(df.query("carat>3"))

In [ ]:
df1 = df.drop(df.query("carat>=3").index)

In [ ]:
# sb.pairplot(df1)

In [ ]:
df1.describe()

In [ ]:
df1.drop(df.query("table > 70 or table < 50 or depth > 70 or depth < 53 or z<2 or z>6").index, inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
df1.describe()

lets replace categorical data with numbers

In [ ]:
df1['color'].values

In [ ]:
df1.replace(['D', 'E', 'F', 'G', 'H', 'I', 'J'], [1, 2, 3, 4, 5, 6, 7], inplace=True)

In [ ]:
df1['clarity'].values

In [ ]:
df1.replace(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1'], [1, 2, 3, 4, 5, 6, 7, 8], inplace=True)

In [ ]:
df1['cut'].values

In [ ]:
df1.replace(['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], [1, 2, 3, 4, 5], inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
df1.describe()

In [ ]:
# sb.pairplot(df1)

filter carat and clarity

In [ ]:
df2 = df1.drop(df1.query("carat>1.7 and (clarity==1 or clarity==2)").index)

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
df2.describe()

In [ ]:
# sb.pairplot(df2)

lets add a column of volume, maybe it can help

In [ ]:
df2['volume'] = df2['x']*df2['y']*df2['z'] 

In [ ]:
df2.head()

In [ ]:
df2.describe()

In [ ]:
# sb.pairplot(df2)

In [ ]:
df3 = df2.drop(df2.query("(carat>2.0 and y<7) or (carat>1.0 and y<6)").index)

In [ ]:
df3.head()

In [ ]:
df3.describe()

In [ ]:
df3.info()

In [ ]:
# sb.pairplot(df3)

after preprocessing from above can be seen that for carat, x, y, z there is several modalities that behave the same. 

In [ ]:
from matplotlib import pyplot as plt
fig, axes = plt.subplots(1, 5, figsize=(16,8))
sb.histplot(df3['carat'], ax=axes[0])
sb.histplot(df3['x'], ax=axes[1])
sb.histplot(df3['y'], ax=axes[2])
sb.histplot(df3['z'], ax=axes[3])
sb.histplot(df3['volume'], ax=axes[4])



In [ ]:
len(df3.query("carat>2"))

In [ ]:
len(df3.query("2>carat>1.5"))

In [ ]:
sb.histplot(df3['carat'])

In [ ]:
sb.histplot(df3['volume'])

Can be seen from volume and carat params that there are 5 subproblems. lets divide them:

In [ ]:
df31 = df3.query("carat<0.5 and price<1500").copy()

In [ ]:
df31.describe()

In [ ]:
sb.displot(df31.price)

In [ ]:
df32 = df3.query("carat>0.5 and carat<0.6 and price < 2700").copy()

In [ ]:
df32.describe()

In [ ]:
sb.displot(df32.price)

In [ ]:
df33 = df3.query("carat>0.6 and carat<1.2 and price < 9000").copy()

In [ ]:
df33.describe()

In [ ]:
sb.displot(df33.price)

In [ ]:
df34 = df3.query("carat>1.2 and volume < 300").copy()

In [ ]:
df34.describe()

In [ ]:
sb.displot(df34.price)

In [ ]:
df35 = df3.query("volume>300 and volume < 400 and price > 8000").copy()

In [ ]:
df35.describe()

In [ ]:
sb.displot(df35.price)

start preprocessing of the data before fit

In [ ]:
# sb.pairplot(df31)

In [ ]:
df31.drop(df31.query("volume > 87 or x > 6 or z<2.2").index, inplace=True)
# sb.pairplot(df31)

In [ ]:
# sb.pairplot(df32)

In [ ]:
df32.drop(df32.query("y > 6 or z<2.5").index, inplace=True)
# sb.pairplot(df32)

In [ ]:
# sb.pairplot(df33)

In [ ]:
df33.drop(df33.query("y < 5").index, inplace=True)
# sb.pairplot(df33)

In [ ]:
# sb.pairplot(df34)

In [ ]:
# sb.pairplot(df35)

In [ ]:
df31.corr()

In [ ]:
df32.corr()

In [ ]:
df33.corr()

In [ ]:
df34.corr()

In [ ]:
df35.corr()

prelminary summary:
    df31: looks like a good corralation - promissing
    df32: looks like a strong corralation with clarity. thinking about oversampling to correct price distribution
    df33: the price distribution looks like bimodel, but the corralation with other features so good might work as well
    df34: the corralation looks promissing
    df35: there is a weak corralation between price and other features - cannot predict prices for volume 300->400. may caused vy not enough data
        

create models:

In [ ]:
import sklearn.ensemble as se
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
import pickle

df31:

In [ ]:
df31.head()

In [ ]:
X = df31.drop('price', axis=1).copy()
y = df31.price

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
sclr = ColumnTransformer([
        ('somename', StandardScaler(), ['carat', 'table', 'depth','x', 'y', 'z', 'volume'])
    ], remainder='passthrough')

X_train_s = sclr.fit_transform(X_train)
X_test_s = sclr.fit_transform(X_test)
X_train_s[1, :]

In [ ]:
model = se.RandomForestRegressor()
model.fit(X_train_s, y_train)
filename = 'df31model.sav'
pickle.dump(model, open('../models/' + filename, 'wb'))
model.score(X_test_s, y_test)


In [ ]:
y_pred = model.predict(X_test_s)
mean_absolute_error(y_pred, y_test)
# y_test.head()

df32:

In [ ]:
df32.head()

In [ ]:
X = df32.drop('price', axis=1).copy()
y = df32.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

sclr = ColumnTransformer([
        ('somename', StandardScaler(), ['carat', 'table', 'depth','x', 'y', 'z', 'volume'])
    ], remainder='passthrough')

X_train_s = sclr.fit_transform(X_train)
X_test_s = sclr.fit_transform(X_test)
X_train_s[1, :]

model = se.RandomForestRegressor()
model.fit(X_train_s, y_train)
filename = 'df32model.sav'
pickle.dump(model, open('../models/' + filename, 'wb'))
model.score(X_test_s, y_test)


In [ ]:
y_pred = model.predict(X_test_s)
mean_absolute_error(y_pred, y_test)

df33:

In [ ]:
df33.head()

In [ ]:
X = df33.drop('price', axis=1).copy()
y = df33.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

sclr = ColumnTransformer([
        ('somename', StandardScaler(), ['carat', 'table', 'depth','x', 'y', 'z', 'volume'])
    ], remainder='passthrough')

X_train_s = sclr.fit_transform(X_train)
X_test_s = sclr.fit_transform(X_test)
X_train_s[1, :]

model = se.RandomForestRegressor()
model.fit(X_train_s, y_train)
model.score(X_test_s, y_test)

In [ ]:
y_pred = model.predict(X_test_s)
mean_absolute_error(y_pred, y_test)

df34:

In [ ]:
df34.head()

In [ ]:
X = df34.drop('price', axis=1).copy()
y = df34.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

sclr = ColumnTransformer([
        ('somename', StandardScaler(), ['carat', 'table', 'depth','x', 'y', 'z', 'volume'])
    ], remainder='passthrough')

X_train_s = sclr.fit_transform(X_train)
X_test_s = sclr.fit_transform(X_test)
X_train_s[1, :]

model = se.RandomForestRegressor()
model.fit(X_train_s, y_train)
model.score(X_test_s, y_test)

In [ ]:
y_pred = model.predict(X_test_s)
mean_absolute_error(y_pred, y_test)

In [ ]:
sb.histplot(df34.x)

In [ ]:
sb.histplot(df34.volume)

df33 and df34 must be seperated for sub tasks  

In [ ]:
# df341 = df34.drop(df34.query("volume < 225").index)
# df342 = df34.drop(df34.query("volume > 225").index)
df341 = df34.drop(df34.query("volume < 225").index).copy()
df342 = df34.drop(df34.query("volume >= 225").index).copy()

In [ ]:
# sb.pairplot(df341)

In [ ]:
df341.drop(df341.query("y<7.0 or carat > 1.8 ").index, inplace=True)
# sb.pairplot(df341)

In [ ]:
# sb.pairplot(df342)

In [ ]:
df342.drop(df342.query("volume < 180  or carat > 1.45").index, inplace=True)
# sb.pairplot(df342)

In [ ]:
X = df342.drop(['price', 'volume'], axis=1).copy()
y = df342.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

sclr = ColumnTransformer([
        ('somename', StandardScaler(), ['carat', 'table', 'depth','x', 'y', 'z'])
    ], remainder='passthrough')

X_train_s = sclr.fit_transform(X_train)
X_test_s = sclr.fit_transform(X_test)
X_train_s[1, :]

model = se.RandomForestRegressor()
model.fit(X_train_s, y_train)
filename = 'df342model.sav'
pickle.dump(model, open('../models/' + filename, 'wb'))
model.score(X_test_s, y_test)


In [ ]:
y_pred = model.predict(X_test_s)
mean_absolute_error(y_pred, y_test)

In [ ]:
df341.describe()

In [ ]:
X = df341.drop(['price', 'volume'], axis=1).copy()
y = df341.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

sclr = ColumnTransformer([
        ('somename', StandardScaler(), ['carat', 'table', 'depth','x', 'y', 'z'])
    ], remainder='passthrough')

X_train_s = sclr.fit_transform(X_train)
X_test_s = sclr.fit_transform(X_test)
X_train_s[1, :]

model = se.RandomForestRegressor()
model.fit(X_train_s, y_train)
filename = 'df341model.sav'
pickle.dump(model, open('../models/' + filename, 'wb'))
model.score(X_test_s, y_test)


In [ ]:
y_pred = model.predict(X_test_s)
mean_absolute_error(y_pred, y_test)

In [ ]:
df33.describe()

In [ ]:
sb.histplot(df33.volume)

In [ ]:
sb.histplot(df33.x)

In [ ]:
df331 = df33.drop(df33.query("x<6").index).copy()
df332 = df33.drop(df33.query("x>=6").index).copy()

In [ ]:
sb.pairplot(df331)

In [ ]:
df331.corr()

In [ ]:
df331.describe()

In [ ]:
df331.drop(df331.query("z<3.5 or x>7.0 or y<6.0").index, inplace=True)
# sb.pairplot(df331)

In [ ]:
sb.pairplot(df332)

In [ ]:
df332.corr()

In [ ]:
df332.describe()

In [ ]:
sb.histplot(df332.volume)

In [ ]:
df332.drop(df332.query("z > 4.0 or x<5.4 or y<5.3 or volume < 110 or volume>120 or table>65").index, inplace=True)
sb.pairplot(df332)

In [ ]:
sb.histplot(df332.volume)

In [ ]:
X = df331.drop(['price', 'depth', 'cut'], axis=1).copy()
y = df331.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

sclr = ColumnTransformer([
        ('somename', StandardScaler(), ['carat','table','color','x', 'y', 'z', 'volume'])
    ], remainder='passthrough')

X_train_s = sclr.fit_transform(X_train)
X_test_s = sclr.fit_transform(X_test)
X_train_s[1, :]

model = se.RandomForestRegressor()
model.fit(X_train_s, y_train)
filename = 'df331model.sav'
pickle.dump(model, open('../models/' + filename, 'wb'))
model.score(X_test_s, y_test)


In [ ]:
y_pred = model.predict(X_test_s)
mean_absolute_error(y_pred, y_test)

In [ ]:
X = df332.drop(['price'], axis=1).copy()
y = df332.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

sclr = ColumnTransformer([
        ('somename', StandardScaler(), ['carat', 'table', 'depth','x', 'y', 'z', 'volume'])
    ], remainder='passthrough')

X_train_s = sclr.fit_transform(X_train)
X_test_s = sclr.fit_transform(X_test)
X_train_s[1, :]

model = se.RandomForestRegressor()
model.fit(X_train_s, y_train)
filename = 'df332model.sav'
pickle.dump(model, open('../models/' + filename, 'wb'))
model.score(X_test_s, y_test)


In [ ]:
y_pred = model.predict(X_test_s)
mean_absolute_error(y_pred, y_test)